In [29]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import torchvision
import torchvision.transforms as transforms

matplotlib.rcParams['figure.figsize'] = (20,10)

In [2]:
data = np.load('../data/cullpdb+profile_5926.npy')
data = data.reshape((5926,700,57))

In [3]:
data.shape
x = data[:,:,:22]
y = data[:,:,22:33] # (brug 35 hvis du vil have solvent properties med)
print(x.shape)
print(y.shape)

(5926, 700, 22)
(5926, 700, 11)


In [4]:
x_train = x[:5430]
y_train = y[:5430]

x_test = x[5435:5690]
y_test = y[5435:5690]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(5430, 700, 22)
(5430, 700, 11)
(255, 700, 22)
(255, 700, 11)


In [5]:
BATCH_SIZE = 32

torch_x_train = torch.from_numpy(x_train).type(torch.ByteTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.ByteTensor) # data type is byte

# create feature and targets tensor for test set.
torch_x_test = torch.from_numpy(x_test).type(torch.ByteTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.ByteTensor) # data type is byte

In [38]:
torch_x_train = torch_x_train.view(-1, 700, 22).float()
torch_x_test = torch_x_test.view(-1, 700, 22).float()
print(torch_x_train.shape)
print(torch_x_test.shape)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_x_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_x_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

torch.Size([5430, 700, 22])
torch.Size([255, 700, 22])


In [52]:
m = nn.Conv1d(700, 300, 5)
#input = torch.randn(10, 700, 22)
#print(input.shape)
it = iter(train_loader)
X_batch, y_batch = next(it)
print(X_batch.shape)
output = m(X_batch)
print(output.shape)

torch.Size([32, 700, 22])
torch.Size([32, 300, 18])


In [51]:
class CNN_new(nn.Module):
    def __init__(self):
        super(CNN_new, self).__init__()
        self.conv1 = nn.Conv1d(700,300,5)
        self.conv2 = nn.Conv1d(300,200,5)
        self.linear_layer1 = nn.Linear(200, 120)
        self.linear_layer2 = nn.Linear(120, 18)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))    
        x = F.relu(self.conv2(x))
        x = x.view(-1, 200)
        x = F.relu(self.linear_layer1(x))
        x = F.relu(self.linear_layer2(x))
        return x
        
cnn = CNN_new()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN_new(
  (conv1): Conv1d(700, 300, kernel_size=(5,), stride=(1,))
  (conv2): Conv1d(300, 200, kernel_size=(5,), stride=(1,))
  (linear_layer1): Linear(in_features=200, out_features=120, bias=True)
  (linear_layer2): Linear(in_features=120, out_features=18, bias=True)
)
torch.Size([448, 18])


In [40]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(22, 32, kernel_size=5)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv1d(22, 32, kernel_size=(5,), stride=(1,))
  (conv2): Conv1d(32, 32, kernel_size=(5,), stride=(1,))
  (conv3): Conv1d(32, 64, kernel_size=(5,), stride=(1,))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


RuntimeError: Given groups=1, weight of size [64, 32, 5], expected input[32, 16, 346] to have 32 channels, but got 16 channels instead